# Doppler Velocity Dealiasing with Py-ART and BALTRAD

In this notebook Doppler Velocity data from the ARM C-band SAPR radar is read using Py-ART and dealiased using BALTRAD.

In [ ]:
%matplotlib inline

Import the necessary modules

In [ ]:
import pyart
import baltrad_pyart_bridge as bridge  # routines to pass data from Py-ART to BALTRAD
import _dealias # BALTRAD's dealiasing module

Read in the data using Py-ART

In [ ]:
radar = pyart.io.read('data/sgpcsaprppi_20110520095101.nc')

Examine the velocity data using Py-ART Display object.

In [ ]:
display = pyart.graph.RadarDisplay(radar)
nyquist_velocity = radar.instrument_parameters['nyquist_velocity']['data'][0]
display.plot_ppi('velocity', 1, colorbar_label='m/s', 
                 vmin=-nyquist_velocity, vmax=nyquist_velocity)

Convert the radar data into a RaveIO object with the velocity data having the correct name.

In [ ]:
vel_data = radar.fields['velocity']['data']
radar.add_field_like('velocity', 'VRAD', vel_data)
rio = bridge.radar2raveio(radar)

Perform Doppler velocity dealiasing using BALTRAD.

In [ ]:
ret = _dealias.dealias(rio.object)
print("This first scan is dealiased:"), _dealias.dealiased(rio.object.getScan(0))

Add the dealiased velocity field to the origin Py-ART radar object.

In [ ]:
temp = bridge.raveio2radar(rio)
if 'dealiased_velocity' in radar.fields:
    radar.fields.pop('dealiased_velocity')
radar.add_field_like('velocity', 'dealiased_velocity', temp.fields['VRAD']['data'])

Plot the dealiased velocities.

In [ ]:
display.plot_ppi('dealiased_velocity', 1, colorbar_label='m/s', 
                 vmin=-2*nyquist_velocity, vmax=2*nyquist_velocity)